In [1]:
import os
import pathlib

from datasets import load_dataset, concatenate_datasets
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import wandb

from hf_wrapper import GPTForSequenceClassification
from tokenizer import load_tokenizer
from utils import flatten_multi_features, load_random_from_pretrained_model, compute_metrics

In [2]:
normal_checkpoint_location = pathlib.Path('./cache/checkpoints/russian_polish_normal_12_5_50k/ckpt.pt')
ipa_checkpoint_location = pathlib.Path('./cache/checkpoints/russian_polish_ipa_12_5_50k/ckpt.pt')
hf_cache = pathlib.Path('./cache')
training_checkpoints = pathlib.Path('./cache/checkpoints')
tokenizer_prefix = pathlib.Path('./cache/tokenizers')
ipa_tokenizer_prefix = 'bpe-rus-pol-ipa-number-preservation'
normal_tokenizer_prefix = 'bpe-rus-pol-normal-number-preservation'

dataset_name = 'iggy12345/russian-xnli-ipa-rosetta'

epochs = 3
context_size = 1024
batch_size = 16
learning_rate = 2e-5

In [3]:
def load_and_preprocess(suffix, split, tokenizer):
    ds = load_dataset(dataset_name, split=split, cache_dir=str(hf_cache))
    fields = [
        'hypothesis' if suffix == 'normal' else f'hypothesis-{suffix}',
        'premise' if suffix == 'normal' else f'premise-{suffix}'
    ]

    def preprocess(examples):
        features = flatten_multi_features(examples, fields)
        encoded = tokenizer(features, truncation=True, max_length=context_size)
        encoded['label'] = examples['label']
        return encoded

    return ds.map(preprocess, batched=True, num_proc=os.cpu_count())

In [4]:
def finetune_transcription(suffix: str):
    ipa = suffix == 'normal'

    checkpoint = ipa_checkpoint_location if ipa else normal_checkpoint_location

    project_name = f"debug-russian-rosetta-small-finetuning-xnli-random-initial"
    temporary_output_dir = training_checkpoints / f"{project_name}/"
    temporary_output_dir.mkdir(parents=True, exist_ok=True)

    vocab_path = tokenizer_prefix / f'{ipa_tokenizer_prefix if ipa else normal_tokenizer_prefix}-vocab.json'
    merges_path = tokenizer_prefix / f'{ipa_tokenizer_prefix if ipa else normal_tokenizer_prefix}-merges.txt'
    tokenizer = load_tokenizer(vocab_path, merges_path)

    base_model = load_random_from_pretrained_model(checkpoint, 'cuda')
    base_model.config.pad_token_id = tokenizer.pad_token_id
    base_model.config.padding_side = tokenizer.padding_side
    model = GPTForSequenceClassification(base_model, num_classes=3).to('cuda')

    train_dataset = load_and_preprocess(suffix, 'train', tokenizer)
    eval_dataset = load_and_preprocess(suffix, 'validation', tokenizer)

    training_args = TrainingArguments(
        eval_strategy="steps",
        eval_steps=1000,
        output_dir=str(temporary_output_dir),
        save_strategy='steps',
        save_steps=1000,
        metric_for_best_model="precision",
        load_best_model_at_end=True,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        logging_steps=100,
        fp16=True,
        warmup_ratio=0.3,
        save_safetensors=False,
        # disable_tqdm=True,
    )

    wrun = wandb.init(entity='aaronjencks-the-ohio-state-university', project=project_name, name=suffix)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    print(f"Training model on {suffix}")
    trainer.train()

    print(f"Final evaluation on {suffix}")
    results = trainer.evaluate()
    print(results)

    wrun.finish()


In [5]:
for suffix in ["normal", "phonemizer", "epitran", "goruut"]:
    finetune_transcription(suffix)

number of parameters: 123.35M


wandb: Currently logged in as: aaronjencks (aaronjencks-the-ohio-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/tmp/ipykernel_1259357/279336246.py:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Training model on normal


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1000,1.117600,1.126177,0.346586,0.347070,0.346586,0.326792
2000,1.110800,1.116790,0.340964,0.343917,0.340964,0.298535
3000,1.097500,1.100971,0.382329,0.380973,0.382329,0.366453
4000,1.096600,1.091415,0.386345,0.382336,0.386345,0.377602
5000,1.073900,1.104519,0.395984,0.429739,0.395984,0.378754
6000,1.042400,1.071045,0.424900,0.440293,0.424900,0.406962
7000,1.053800,1.045136,0.438554,0.458285,0.438554,0.409338
8000,1.027300,1.028360,0.461446,0.473191,0.461446,0.463879
9000,1.022900,1.019781,0.473092,0.507633,0.473092,0.455885
10000,1.012600,1.016210,0.486747,0.486057,0.486747,0.485747


...

Training model on goruut


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1000,1.148200,1.148797,0.341365,0.342031,0.341365,0.337475
2000,1.115700,1.129809,0.332530,0.334253,0.332530,0.321200
3000,1.113900,1.112930,0.342972,0.345284,0.342972,0.341261
4000,1.116400,1.109340,0.351004,0.350062,0.351004,0.342580
5000,1.107900,1.099129,0.373494,0.375049,0.373494,0.371561
6000,1.095300,1.093369,0.381526,0.377712,0.381526,0.375965
7000,1.085500,1.095535,0.383133,0.387394,0.383133,0.366837
8000,1.078200,1.082809,0.410040,0.416262,0.410040,0.393703
9000,1.070000,1.076824,0.424096,0.431363,0.424096,0.382152
10000,1.062900,1.060483,0.420482,0.418383,0.420482,0.415115


Final evaluation on goruut


{'eval_loss': 0.9838324189186096, 'eval_accuracy': 0.5140562248995983, 'eval_precision': 0.5362580400252271, 'eval_recall': 0.5140562248995983, 'eval_f1': 0.5157914915374591, 'eval_runtime': 4.0921, 'eval_samples_per_second': 608.494, 'eval_steps_per_second': 38.123, 'epoch': 3.0}


eval/accuracy,▁▁▁▂▃▄▅▅▆▅▅▅▆▆▇▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇█▇█████
eval/f1,▂▁▂▂▃▃▄▄▆▆▅▄▇▆▆▆▇▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇█▇██████
eval/loss,██▇▆▅▄▄▅▃▅▃▃▃▃▂▄▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁
eval/precision,▁▁▁▂▂▄▄▆▆▆▅▅▇▅▆▅▇▇▅▆▇▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
eval/recall,▁▁▂▃▃▄▅▆▅▆▅▅▆▆▆▆▇▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
eval/runtime,▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▆▆█▇▂▄▄▃▁▃
eval/samples_per_second,▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▁▃▃▃▃▃▃▃▃▃▃▃▃▂█▆▅▆▇
eval/steps_per_second,▃▃▃▃▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▂▃▃▂▂▂▃▁▃▂▇▆▅▆██▆
train/epoch,▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,▅▅▅▅▄▆▅▅▄▁█▂▂▄▅▂▅▃▆▂▄▄▁▂▄▁▃▆▂▄▄▂▃▄▃▁▂▄▁▁
